# Volumetric Quantifications

This notebook demonstrates volumetric quantifications. It assumes you have properly [setup](./README.md) your environment and ran the introductory examples. Here's what we will accomplish:

1. Generate a convex hull of the axon terminals within a specific brain region
2. Compare the volume of this convex hull to the volume of the encompasing Allen CCF compartment.
3. Do PCA on the point cloud of the relevant axon terminals.
4. Visualize the results of these operations using [Reconstruction Viewer](https://imagej.net/SNT:_Reconstruction_Viewer).

## Imports
We'll need to 1) import Python modules; 2) initalize ij from local Fiji installation, and 3) import all relevant SNT (Java) classes: [AllenCompartment](https://morphonets.github.io/SNT/index.html?sc/fiji/snt/annotation/AllenCompartment.html), [AllenUtils](https://morphonets.github.io/SNT/index.html?sc/fiji/snt/annotation/AllenUtils.html), [MouseLightLoader](https://morphonets.github.io/SNT/index.html?sc/fiji/snt/io/MouseLightLoader.html), [PointInImage](https://morphonets.github.io/SNT/index.html?sc/fiji/snt/util/PointInImage.html), [Tree](https://morphonets.github.io/SNT/index.html?sc/fiji/snt/Tree.html), [TreeAnalyzer](https://morphonets.github.io/SNT/index.html?sc/fiji/snt/analysis/TreeAnalyzer.html), [Viewer3D](https://morphonets.github.io/SNT/index.html?sc/fiji/snt/viewer/Viewer3D.html):

In [1]:
import os
import sys
import ijfinder
import imagej
import numpy as np
from collections import defaultdict

fiji_path = ijfinder.getpath().decode('utf-8')
if os.path.isdir(fiji_path):
    ij = imagej.init(fiji_path, headless=False)
else:
    print("Cannot proceed: Fiji not found!")

from jnius import autoclass, cast
AllenCompartment = autoclass('sc.fiji.snt.annotation.AllenCompartment')
AllenUtils = autoclass('sc.fiji.snt.annotation.AllenUtils')
MouseLightLoader = autoclass('sc.fiji.snt.io.MouseLightLoader')
NodeStatistics = autoclass('sc.fiji.snt.analysis.NodeStatistics')
PointInImage = autoclass('sc.fiji.snt.util.PointInImage')
Tree = autoclass('sc.fiji.snt.Tree')
TreeAnalyzer = autoclass('sc.fiji.snt.analysis.TreeAnalyzer')
Viewer3D = autoclass('sc.fiji.snt.viewer.Viewer3D')

We'll define two support functions: one to download the axonal arbor of a MouseLight neuron, the other to detect the brain area that is the most innervated by its axon:

In [2]:
def get_axon(id_string):
    """Fetchs an axonal arbor from the MouseLight database by ID"""
    loader = MouseLightLoader(id_string)
    if not loader.isDatabaseAvailable():
        print("Could not connect to ML database", "Error")
        return null
    if not loader.idExists():
        print("Somewhow the specified id was not found", "Error")
        return null
    # Extract the axon sub-tree
    return loader.getTree("axon")


def get_compartment_terminals(tree):
    """Finds the AllenCompartment containing the largest number 
    of axon terminal nodes and returns a collection containing 
    these nodes as well as the id of the relevant AllenCompartment"""
    
    # Use TreeAnalyzer to extract the terminal nodes from the Tree.
    # Instantiate a NodeStatistics instance and retrieve a list of the endpoints for
    # each target brain region (a BrainAnnotation) in a dictionary, where the keys are
    # the brain annotations. Since this neuron was fetched from the MouseLight database,
    # the annotations are instances of the AllenCompartment Class
    # https://morphonets.github.io/SNT/sc/fiji/snt/annotation/BrainAnnotation.html
    tips = TreeAnalyzer(tree).getTips()
    node_stats = NodeStatistics(tips)
    compartment_dict = ij.py.from_java(node_stats.getAnnotatedNodes())

    # Get the compartment containing the maximum number of axon terminals
    max_compartment = max(compartment_dict, key= lambda x: len(compartment_dict[x]))
    # Get the associated list of terminals.
    compartment_tips = compartment_dict[max_compartment]
    
    return compartment_tips, max_compartment

In [3]:
tree_axon = get_axon('AA1044')
axon_terminals, compartment = get_compartment_terminals(tree_axon)

# We can get the dominant hemi-half of the target compartment using AllenUtils.
axon_terminals_coords = [[t.getX(), t.getY(), t.getZ()] for t in axon_terminals]
centroid = np.mean(axon_terminals_coords, axis=0)
hemisphere = "left" if AllenUtils.isLeftHemisphere(centroid[0], centroid[1], centroid[2]) else "right"
print("Dominant hemisphere: " + hemisphere)

# Now we may begin adding the computed objects to SNT's Viewer3D.
# Viewer3D has a script-friendly 'add' method which accepts a variety of differnent objects,
# e.g., Tree, AbstractDrawable, OBJMesh, etc...
viewer = Viewer3D()
viewer.add(tree_axon)

# Get the OBJMesh which represents the AllenCompartment instance.
# Most meshes have pre-computed volumes (via surface integrals) associated with them.
# However, certain meshes (e.g., third ventricle) are not watertight,
# which precludes a direct volume calculation. 
# Still, it is possible to approximate the volume of these compartments
# using the convex hull computed by Viewer3D.
# https://morphonets.github.io/SNT/sc/fiji/snt/viewer/OBJMesh.html
obj_mesh = compartment.getMesh()

# Add the compartment mesh, which contains both left and right nuclei.
viewer.add(obj_mesh)

# And the convex hull
axon_hull = viewer.annotateSurface(ij.py.to_java(axon_terminals), 
                                   "Convex Hull of Axon Terminals within {}".format(compartment.name()), 
                                   True) # compute hull volume

axon_hull.setColor("orange", 95) # transparency (%)

# Now compare the volumes of the convex hulls
# Since this compartment mesh is composed of both hemi-halves, we can approximate
# the volume of one hemi-half by dividing the total mesh volume by 2 (assuming perfect symmetry).
print("Percentage of volume occupied by the convex hull of "
      "the axon terminals with respect to the {} Caudoputamen".format(hemisphere))
print((axon_hull.getVolume() / (0.5 * obj_mesh.getVolume())) * 100, "%")

# Finally, we can visualize all our hard work!
viewer.show()
viewer.setAnimationEnabled(True)

#To embed the snapshot in this notebook
#snapshot_path = os.getcwd() + '/images/convexhull1.png'
#viewer.saveSnapshot(snapshot_path)
#from IPython.display import Image, display
#display(Image(filename=snapshot_path))

Dominant hemisphere: right
Percentage of volume occupied by the convex hull of the axon terminals with respect to the right Caudoputamen
4.130456560628865 %


![](./images/convexhull.png)

As a bonus, let's estimate the principal components of the covariance on the point cloud given by the axon terminals and annotate the resulting eigenvectors as line segments.

In [4]:
# First, subtract the mean from the points.
points = np.copy(axon_terminals_coords)
points -= centroid
# Compute the eigenvalues and eigenvectors of the covariance matrix.
e_values, e_vectors = np.linalg.eig(np.cov(points.transpose()))

# Construct the line segments using the eigenvectors.
viewer.setAnimationEnabled(False)
viewer.setSceneUpdatesEnabled(False)
for i in range(e_vectors.shape[1]):
    # The line segments will originate at the centroid of the terminals.
    end = centroid + ((np.sqrt(e_values[i]) * 10) * e_vectors[:, i])
    line_segment = [PointInImage(centroid[0], centroid[1], centroid[2]), PointInImage(end[0], end[1], end[2])]
    # Viewer3D supports adding annotations of various types, and allows customization of 
    # their visual properties.
    # https://morphonets.github.io/SNT/sc/fiji/snt/viewer/Annotation3D.html
    annot = viewer.annotateLine(ij.py.to_java(line_segment), "component {}".format(i))
    annot.setColor("white", 10)
    annot.setSize(20)

viewer.setSceneUpdatesEnabled(True)
viewer.updateView()